# AAI-520 Final Project: Autonomous Investment Research Agent

**Powered by Agentic AI with Multi-Workflow Patterns**

This notebook demonstrates an autonomous investment research agent that:
1. Plans its research steps for a given stock
2. Uses tools dynamically (APIs, datasets, retrieval)
3. Self-reflects to assess output quality
4. Learns across runs (memory/notes for improvement)

## Three Core Workflow Patterns
- **Prompt Chaining**: Sequential processing pipeline (Ingest → Preprocess → Classify → Extract → Summarize)
- **Routing**: Intelligent task distribution to specialist agents
- **Evaluator-Optimizer**: Iterative self-improvement through evaluation and refinement

## 1. Setup and Imports

In [ ]:
# Core imports
import os
import sys
import time
import json
import requests
from typing import Dict, List, Optional, Literal, Set
from textwrap import dedent

# Pydantic for data models
from pydantic import BaseModel, Field

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.tools import DuckDuckGoSearchResults, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# NetworkX for knowledge graphs
import networkx as nx

print("✓ All imports loaded successfully")

## 2. Configuration

In [ ]:
# LLM Configuration
LMSTUDIO_URL = "http://localhost:1234/v1"
LLM_NAME = "gemma-3-27b-it-qat"  # Match the actual model ID from LM Studio
ENTITY_EXTRACTOR = "gemma-3-27b-it-qat"  # Use same model for extraction
EXTRACTION_TEMPERATURE = 0

# API Keys
ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Data Fetching
FETCH_DELAY_SECONDS = 1.5

# Database
SQLITE_DB_PATH = "kg_store.db"
MEMORY_FILE = "agent_memory.txt"

# Financial Entity Labels
FINANCIAL_ENTITY_LABELS = {
    "ORGANIZATION", "PERSON", "PRODUCT", "EVENT",
    "STOCK_SYMBOL", "POLICY", "GOVERNMENT", "COMPANY"
}

print("✓ Configuration loaded")
print(f"  LLM: {LLM_NAME}")
print(f"  LM Studio URL: {LMSTUDIO_URL}")

## 3. Data Models

In [ ]:
# === Entity Models ===
class Entity(BaseModel):
    """Named entity extracted from text."""
    text: str = Field(description="The extracted entity text")
    label: str = Field(description="Entity type (PERSON, ORGANIZATION, STOCK_SYMBOL, etc.)")
    confidence: Optional[float] = Field(description="Confidence score", default=None)


class NERResponse(BaseModel):
    """Named Entity Recognition response."""
    entities: List[Entity] = Field(description="List of extracted named entities")


# === Prompt Chaining Models ===
class NewsArticle(BaseModel):
    """Raw news article model."""
    title: str
    content: str
    source: str
    timestamp: Optional[str] = None


class PreprocessedNews(BaseModel):
    """Cleaned and normalized news article."""
    title: str
    cleaned_content: str
    source: str
    word_count: int
    contains_financials: bool


class NewsClassification(BaseModel):
    """Classification result for news article."""
    category: str = Field(description="Category: earnings, market_analysis, policy, merger_acquisition, general")
    sentiment: str = Field(description="Sentiment: positive, negative, neutral")
    relevance_score: float = Field(description="Relevance score 0-1")
    confidence: float = Field(description="Classification confidence 0-1")


class ExtractedEntities(BaseModel):
    """Extracted entities and key facts from news."""
    companies: List[str] = Field(description="Company names mentioned")
    people: List[str] = Field(description="Key people mentioned")
    financial_metrics: List[str] = Field(description="Financial metrics (revenue, EPS, etc.)")
    key_events: List[str] = Field(description="Important events or announcements")
    stock_symbols: List[str] = Field(description="Stock ticker symbols")


class NewsSummary(BaseModel):
    """Final summarized output."""
    executive_summary: str = Field(description="2-3 sentence summary")
    key_points: List[str] = Field(description="Bullet points of key takeaways")
    investment_implications: str = Field(description="What this means for investors")
    entities: ExtractedEntities
    classification: NewsClassification


# === Routing Models ===
class RoutingDecision(BaseModel):
    """Router's decision on where to send content."""
    route: Literal["earnings", "news", "market"] = Field(
        description="Which specialist to route to: earnings, news, or market"
    )
    confidence: float = Field(description="Confidence in routing decision (0-1)")
    reasoning: str = Field(description="Brief explanation of routing choice")


class EarningsAnalysis(BaseModel):
    """Output from the earnings specialist."""
    revenue_analysis: str = Field(description="Analysis of revenue performance")
    profitability_analysis: str = Field(description="Analysis of profit margins and EPS")
    growth_trends: str = Field(description="YoY and QoQ growth trends")
    guidance_assessment: str = Field(description="Assessment of forward guidance")
    key_metrics: List[str] = Field(description="Key financial metrics extracted")
    recommendation: str = Field(description="Investment recommendation")


class NewsAnalysis(BaseModel):
    """Output from the news specialist."""
    event_summary: str = Field(description="Summary of the news event")
    market_impact: str = Field(description="Potential market impact analysis")
    stakeholder_analysis: str = Field(description="Who is affected and how")
    timeline: str = Field(description="Timeline of events or expected developments")
    credibility_score: float = Field(description="News source credibility (0-1)")
    actionable_insights: List[str] = Field(description="Actionable takeaways")


class MarketAnalysis(BaseModel):
    """Output from the market specialist."""
    trend_analysis: str = Field(description="Overall market/sector trend analysis")
    technical_indicators: str = Field(description="Key technical signals")
    sentiment_assessment: str = Field(description="Market sentiment evaluation")
    risk_factors: List[str] = Field(description="Identified risk factors")
    opportunities: List[str] = Field(description="Identified opportunities")
    outlook: str = Field(description="Short to medium term outlook")


# === Evaluator-Optimizer Models ===
class InvestmentAnalysis(BaseModel):
    """Investment analysis generated by the analyzer."""
    ticker: str = Field(description="Stock ticker symbol")
    recommendation: str = Field(description="Buy/Hold/Sell recommendation")
    target_price: float = Field(description="Price target")
    investment_thesis: str = Field(description="Core investment thesis")
    key_catalysts: List[str] = Field(description="Key catalysts for the stock")
    risks: List[str] = Field(description="Key risks")
    financial_highlights: str = Field(description="Key financial metrics and trends")
    conclusion: str = Field(description="Summary conclusion")


class QualityEvaluation(BaseModel):
    """Evaluation of analysis quality."""
    overall_score: float = Field(description="Overall quality score 0-100")
    completeness_score: float = Field(description="Completeness 0-100")
    accuracy_score: float = Field(description="Accuracy/logic 0-100")
    actionability_score: float = Field(description="Actionability 0-100")
    
    strengths: List[str] = Field(description="What the analysis does well")
    weaknesses: List[str] = Field(description="What needs improvement")
    missing_elements: List[str] = Field(description="Important missing information")
    
    specific_feedback: str = Field(description="Detailed constructive feedback")
    improvement_suggestions: List[str] = Field(description="Specific suggestions to improve")
    
    is_acceptable: bool = Field(description="Whether analysis meets quality threshold")


# === Agent Models ===
class ResearchPlan(BaseModel):
    """Research plan generated by the agent."""
    ticker: str = Field(description="Stock ticker to research")
    research_steps: List[str] = Field(description="Ordered list of research steps")
    data_sources_needed: List[str] = Field(description="Data sources required")
    estimated_complexity: str = Field(description="low, medium, or high")
    key_questions: List[str] = Field(description="Key questions to answer")


class ResearchReflection(BaseModel):
    """Self-reflection on research quality."""
    completeness_score: float = Field(description="How complete is the research (0-100)")
    confidence_score: float = Field(description="Confidence in findings (0-100)")
    data_quality_score: float = Field(description="Quality of data sources (0-100)")
    
    strengths: List[str] = Field(description="Strong aspects of the research")
    gaps: List[str] = Field(description="Gaps or missing information")
    reliability_concerns: List[str] = Field(description="Data reliability issues")
    
    overall_assessment: str = Field(description="Overall quality assessment")
    improvement_recommendations: List[str] = Field(
        description="How to improve future research"
    )


class ResearchResult(BaseModel):
    """Final research output."""
    ticker: str
    plan: ResearchPlan
    knowledge_graph_summary: str
    news_analysis: Optional[Dict]
    specialized_analysis: Optional[Dict]
    investment_analysis: Optional[Dict]
    reflection: ResearchReflection
    execution_time_seconds: float


print("✓ Data models defined")

## 4. Utility Functions

### 4.1 LLM Factory

In [ ]:
def create_chat_llm(temperature: float = 0.7, model_name: str = None) -> ChatOpenAI:
    """Create a standard chat LLM instance."""
    return ChatOpenAI(
        base_url=LMSTUDIO_URL,
        api_key="dummy",
        model_name=model_name or LLM_NAME,
        temperature=temperature
    )


def create_extraction_llm(temperature: float = 0) -> ChatOpenAI:
    """Create an LLM instance optimized for entity extraction."""
    return ChatOpenAI(
        base_url=LMSTUDIO_URL,
        api_key="dummy",
        model_name=ENTITY_EXTRACTOR,
        temperature=temperature
    )


print("✓ LLM factory functions defined")

### 4.2 Financial Data Fetchers

In [ ]:
class FinancialDataFetcher:
    """Fetches financial data from various sources."""
    
    def __init__(self):
        """Initialize data fetching tools."""
        self.yahoo_news = YahooFinanceNewsTool()
        self.wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        self.ddgo = DuckDuckGoSearchResults()
    
    def fetch_alpha_vantage_quote(self, ticker: str) -> str:
        """Fetch stock quote from Alpha Vantage."""
        try:
            url = "https://www.alphavantage.co/query"
            params = {
                "function": "GLOBAL_QUOTE",
                "symbol": ticker,
                "apikey": ALPHA_VANTAGE_API_KEY
            }
            resp = requests.get(url, params=params, timeout=15)
            data = resp.json()
            quote = data.get("Global Quote") or {}
            
            if not quote:
                return ""
            
            return f"AlphaVantage Quote for {ticker}:\n{json.dumps(quote, indent=2)}"
        except Exception as e:
            print(f"[Error] AlphaVantage: {e}")
            return ""
    
    def fetch_yahoo_news(self, ticker: str) -> str:
        """Fetch recent news from Yahoo Finance."""
        try:
            res = self.yahoo_news.invoke(ticker)
            if isinstance(res, str) and len(res.strip()) > 50 and "No news found" not in res:
                return f"Yahoo Finance News for {ticker}:\n{res}"
        except Exception as e:
            print(f"[Error] YahooNewsTool: {e}")
        return ""
    
    def fetch_wikipedia_info(self, entity_name: str) -> str:
        """Fetch information from Wikipedia."""
        try:
            res = self.wiki.run(entity_name)
            if isinstance(res, str) and len(res.strip()) > 50:
                return res[:4000]  # Limit length
        except Exception as e:
            print(f"[Error] Wikipedia: {e}")
        return ""
    
    def fetch_duckduckgo_search(self, query: str) -> str:
        """Fetch search results from DuckDuckGo."""
        try:
            res = self.ddgo.run(query)
            if isinstance(res, str) and len(res.strip()) > 50:
                return res[:4000]  # Limit length
        except Exception as e:
            print(f"[Error] DuckDuckGo: {e}")
        return ""
    
    def fetch_entity_info(self, entity_name: str, entity_label: str) -> str:
        """Fetch information about an entity based on its type."""
        entity_label = entity_label.upper()
        text = ""
        
        # Stock symbols and companies: get financial data + news
        if entity_label in {"STOCK_SYMBOL", "ORGANIZATION", "COMPANY"}:
            text = self.fetch_alpha_vantage_quote(entity_name)
            text += "\n\n" + self.fetch_yahoo_news(entity_name)
        
        # People: get background information
        elif entity_label == "PERSON":
            text = self.fetch_wikipedia_info(entity_name)
        
        # Policies, government: search for recent news
        elif entity_label in {"POLICY", "GOVERNMENT", "EVENT"}:
            print(f"[Policy Search] Searching for '{entity_name}'")
            query = f"{entity_name} government budget OR regulation OR policy 2025 site:reuters.com OR site:bloomberg.com"
            text = self.fetch_duckduckgo_search(query)
        
        # Fallback: try Wikipedia/DuckDuckGo
        if not text or text.strip() == "":
            wiki_result = self.fetch_wikipedia_info(entity_name)
            if wiki_result:
                text = wiki_result
            else:
                text = self.fetch_duckduckgo_search(entity_name)
        
        return text.strip() or f"No relevant data found for {entity_name}"


print("✓ FinancialDataFetcher class defined")

### 4.3 Knowledge Graph Builder

In [ ]:
def is_relevant_entity(ent: Entity) -> bool:
    """Filter out irrelevant or low-quality entities."""
    # Check minimum length
    if not ent.text or len(ent.text.strip()) < 2:
        return False
    
    # Filter numeric values (prices, percentages)
    if ent.text.replace('.', '', 1).replace('%', '', 1).isdigit():
        return False
    
    # Check if entity type is relevant
    if ent.label.upper() not in FINANCIAL_ENTITY_LABELS:
        return False
    
    return True


class KnowledgeGraphBuilder:
    """Builds and expands a knowledge graph from financial entities."""
    
    def __init__(
        self,
        llm: Optional[ChatOpenAI] = None,
        fetcher: Optional[FinancialDataFetcher] = None
    ):
        """Initialize the knowledge graph builder."""
        self.extraction_llm = llm or create_extraction_llm()
        self.fetcher = fetcher or FinancialDataFetcher()
        self.graph = nx.Graph()
        
        # Set up structured output for NER
        self.ner_model = self.extraction_llm.with_structured_output(NERResponse)
    
    def extract_entities(self, text: str) -> List[Entity]:
        """Extract named entities from text using NER."""
        try:
            prompt = dedent(f"""Extract named entities from the following financial text.

Focus on:
- Companies and organizations
- People (executives, analysts)
- Stock symbols
- Products and services
- Events
- Government/policy entities

Text:
{text[:8000]}

Extract all relevant entities:""")
            
            ner_result = self.ner_model.invoke(prompt)
            relevant_entities = [e for e in ner_result.entities if is_relevant_entity(e)]
            return relevant_entities
        
        except Exception as e:
            print(f"[Error] NER extraction failed: {e}")
            return []
    
    def add_entity_to_graph(
        self,
        entity: Entity,
        related_entities: List[Entity],
        context: Optional[str] = None
    ):
        """Add an entity and its relationships to the graph."""
        # Add main entity node if not exists
        if not self.graph.has_node(entity.text):
            self.graph.add_node(
                entity.text,
                label=entity.label,
                confidence=entity.confidence
            )
        
        # Add related entities and edges
        for related in related_entities:
            if not self.graph.has_node(related.text):
                self.graph.add_node(
                    related.text,
                    label=related.label,
                    confidence=related.confidence
                )
            
            # Add edge if not exists
            if not self.graph.has_edge(entity.text, related.text):
                self.graph.add_edge(
                    entity.text,
                    related.text,
                    relation="mentioned_with",
                    context=(context or "")[:400]
                )
    
    def expand_from_seed(
        self,
        seed_entity: str,
        seed_label: str = "STOCK_SYMBOL",
        depth: int = 2,
        throttle: float = FETCH_DELAY_SECONDS
    ) -> nx.Graph:
        """Expand knowledge graph starting from a seed entity."""
        visited: Set[str] = set()
        current_layer = [Entity(text=seed_entity, label=seed_label)]
        
        print(f"\n=== EXPANDING KNOWLEDGE GRAPH ===")
        print(f"Seed: {seed_entity} ({seed_label})")
        print(f"Depth: {depth} layers")
        
        for layer in range(1, depth + 1):
            print(f"\n--- Layer {layer}/{depth} ---")
            next_layer: List[Entity] = []
            
            for entity in current_layer:
                if entity.text in visited:
                    continue
                
                visited.add(entity.text)
                print(f"[L{layer}] Processing '{entity.text}' ({entity.label})")
                
                # Fetch information about this entity
                info = self.fetcher.fetch_entity_info(entity.text, entity.label)
                
                if not info.strip() or info.startswith("No relevant data"):
                    print(f"[L{layer}] No data found for '{entity.text}'")
                    continue
                
                # Extract entities from the fetched information
                print(f"[L{layer}] Extracting entities from data...")
                discovered_entities = self.extract_entities(info)
                print(f"[L{layer}] Found {len(discovered_entities)} entities")
                
                # Add to graph
                self.add_entity_to_graph(entity, discovered_entities, info)
                
                # Add undiscovered entities to next layer
                for discovered in discovered_entities:
                    if discovered.text not in visited:
                        next_layer.append(discovered)
                
                # Throttle to avoid rate limits
                time.sleep(throttle)
            
            current_layer = next_layer
            
            if not current_layer:
                print(f"\n[L{layer}] No new entities to explore. Stopping.")
                break
        
        print(f"\n=== GRAPH EXPANSION COMPLETE ===")
        print(f"Total nodes: {self.graph.number_of_nodes()}")
        print(f"Total edges: {self.graph.number_of_edges()}")
        
        return self.graph
    
    def get_graph_summary(self) -> str:
        """Generate a text summary of the graph."""
        lines = [
            f"Knowledge Graph Summary:",
            f"  Nodes: {self.graph.number_of_nodes()}",
            f"  Edges: {self.graph.number_of_edges()}",
            f"\nEntities by Type:"
        ]
        
        # Count entities by type
        label_counts: Dict[str, int] = {}
        for node, data in self.graph.nodes(data=True):
            label = data.get('label', 'UNKNOWN')
            label_counts[label] = label_counts.get(label, 0) + 1
        
        for label, count in sorted(label_counts.items(), key=lambda x: x[1], reverse=True):
            lines.append(f"  {label}: {count}")
        
        lines.append(f"\nTop Connected Entities:")
        # Get nodes sorted by degree (number of connections)
        node_degrees = [(node, self.graph.degree(node)) for node in self.graph.nodes()]
        node_degrees.sort(key=lambda x: x[1], reverse=True)
        
        for node, degree in node_degrees[:10]:
            label = self.graph.nodes[node].get('label', 'UNKNOWN')
            lines.append(f"  {node} ({label}): {degree} connections")
        
        return "\n".join(lines)


print("✓ KnowledgeGraphBuilder class defined")

## 5. Workflow Pattern 1: Prompt Chaining

Sequential pipeline: Ingest → Preprocess → Classify → Extract → Summarize

In [ ]:
class PromptChainWorkflow:
    """Implements the Prompt Chaining workflow pattern for news analysis."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        """Initialize the workflow with an LLM instance."""
        self.llm = llm or create_chat_llm(temperature=0.3)
    
    def step1_ingest(self, raw_text: str, source: str = "Unknown") -> NewsArticle:
        """Step 1: Ingest raw news data."""
        lines = raw_text.strip().split('\n')
        title = lines[0] if lines else "Untitled"
        content = '\n'.join(lines[1:]) if len(lines) > 1 else raw_text
        
        return NewsArticle(
            title=title,
            content=content,
            source=source
        )
    
    def step2_preprocess(self, article: NewsArticle) -> PreprocessedNews:
        """Step 2: Clean and preprocess the news article."""
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a text preprocessing expert. Clean and normalize the following news article. "
                      "Remove ads, boilerplate, redundant info. Keep only substantive content."),
            ("user", "Title: {title}\n\nContent: {content}")
        ])
        
        chain = prompt | self.llm
        response = chain.invoke({
            "title": article.title,
            "content": article.content
        })
        
        cleaned_content = response.content
        word_count = len(cleaned_content.split())
        contains_financials = any(term in cleaned_content.lower()
                                 for term in ['revenue', 'earnings', 'eps', 'profit', 'loss', 'quarter'])
        
        return PreprocessedNews(
            title=article.title,
            cleaned_content=cleaned_content,
            source=article.source,
            word_count=word_count,
            contains_financials=contains_financials
        )
    
    def step3_classify(self, preprocessed: PreprocessedNews) -> NewsClassification:
        """Step 3: Classify the news article by category and sentiment."""
        structured_llm = self.llm.with_structured_output(NewsClassification)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a financial news classifier. Analyze the article and classify it.

Categories:
- earnings: Quarterly/annual earnings reports
- market_analysis: Market trends, sector analysis
- policy: Government policy, regulation, central bank
- merger_acquisition: M&A, partnerships, deals
- general: Other business news

Sentiment: positive, negative, or neutral
Relevance: How relevant to investors (0-1)
Confidence: Your confidence in this classification (0-1)"""),
            ("user", "Title: {title}\n\nContent: {content}")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({
            "title": preprocessed.title,
            "content": preprocessed.cleaned_content
        })
    
    def step4_extract(self, preprocessed: PreprocessedNews) -> ExtractedEntities:
        """Step 4: Extract key entities and facts from the article."""
        structured_llm = self.llm.with_structured_output(ExtractedEntities)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """Extract key entities from this financial news article.

Focus on:
- Company names and stock symbols
- Key executives, analysts, officials
- Financial metrics (revenue, EPS, growth %, margins, etc.)
- Important events or announcements

Be precise and extract only what's explicitly mentioned."""),
            ("user", "{content}")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({"content": preprocessed.cleaned_content})
    
    def step5_summarize(
        self,
        preprocessed: PreprocessedNews,
        classification: NewsClassification,
        entities: ExtractedEntities
    ) -> NewsSummary:
        """Step 5: Generate final summary with investment implications."""
        structured_llm = self.llm.with_structured_output(NewsSummary)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a financial analyst. Create a concise summary for investors.

Include:
- Executive summary (2-3 sentences)
- Key points as bullet list
- Investment implications (what this means for investors)

Be clear, actionable, and focused on financial impact."""),
            ("user", """Article: {title}

Content: {content}

Category: {category}
Sentiment: {sentiment}

Extracted Entities:
Companies: {companies}
People: {people}
Financial Metrics: {metrics}
Key Events: {events}

Create a comprehensive summary.""")
        ])
        
        chain = prompt | structured_llm
        result = chain.invoke({
            "title": preprocessed.title,
            "content": preprocessed.cleaned_content,
            "category": classification.category,
            "sentiment": classification.sentiment,
            "companies": ", ".join(entities.companies) if entities.companies else "None",
            "people": ", ".join(entities.people) if entities.people else "None",
            "metrics": ", ".join(entities.financial_metrics) if entities.financial_metrics else "None",
            "events": ", ".join(entities.key_events) if entities.key_events else "None"
        })
        
        # Attach entities and classification to summary
        result.entities = entities
        result.classification = classification
        
        return result
    
    def run(self, raw_text: str, source: str = "Unknown") -> NewsSummary:
        """Execute the complete prompt chaining workflow."""
        print("\n=== PROMPT CHAINING WORKFLOW ===")
        
        # Step 1: Ingest
        print("\n[1/5] Ingesting raw news...")
        article = self.step1_ingest(raw_text, source)
        print(f"✓ Ingested: {article.title[:50]}...")
        
        # Step 2: Preprocess
        print("\n[2/5] Preprocessing content...")
        preprocessed = self.step2_preprocess(article)
        print(f"✓ Preprocessed: {preprocessed.word_count} words, "
              f"Contains financials: {preprocessed.contains_financials}")
        
        # Step 3: Classify
        print("\n[3/5] Classifying news...")
        classification = self.step3_classify(preprocessed)
        print(f"✓ Classified: {classification.category} | {classification.sentiment} "
              f"(confidence: {classification.confidence:.2f})")
        
        # Step 4: Extract
        print("\n[4/5] Extracting entities...")
        entities = self.step4_extract(preprocessed)
        print(f"✓ Extracted: {len(entities.companies)} companies, "
              f"{len(entities.financial_metrics)} metrics")
        
        # Step 5: Summarize
        print("\n[5/5] Generating summary...")
        summary = self.step5_summarize(preprocessed, classification, entities)
        print(f"✓ Summary complete: {len(summary.key_points)} key points")
        
        print("\n=== WORKFLOW COMPLETE ===\n")
        return summary


print("✓ PromptChainWorkflow class defined")

### Demo: Prompt Chaining

In [ ]:
# Sample news for demonstration
sample_news = """
Microsoft Reports Strong Q4 Earnings, Cloud Revenue Surges 25%

Microsoft Corporation (MSFT) announced its fiscal Q4 2024 results today, beating analyst
expectations with revenue of $61.9 billion, up 15% year-over-year. CEO Satya Nadella
highlighted the company's AI investments, noting that Azure cloud services grew 25%
driven by demand for AI infrastructure.

Earnings per share came in at $2.95, compared to consensus estimates of $2.85. The
Intelligent Cloud segment generated $28.5 billion in revenue, while Productivity and
Business Processes reached $19.6 billion.

"We are seeing unprecedented demand for AI capabilities across our cloud platform,"
Nadella stated during the earnings call. The company also announced a 10% increase
in its quarterly dividend to $0.75 per share.

Shares rose 4% in after-hours trading following the announcement.
"""

# Run the workflow
workflow = PromptChainWorkflow()
summary = workflow.run(sample_news, source="Company Press Release")

# Display results
print("\n=== FINAL SUMMARY ===")
print(f"\nExecutive Summary:\n{summary.executive_summary}")
print(f"\nKey Points:")
for i, point in enumerate(summary.key_points, 1):
    print(f"  {i}. {point}")
print(f"\nInvestment Implications:\n{summary.investment_implications}")
print(f"\nCategory: {summary.classification.category}")
print(f"Sentiment: {summary.classification.sentiment}")
print(f"Companies: {', '.join(summary.entities.companies)}")

## 6. Workflow Pattern 2: Routing

Routes content to specialized analyst agents based on content type

In [ ]:
class ContentRouter:
    """Routes incoming content to specialized analyst agents."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        """Initialize the router with an LLM."""
        self.llm = llm or create_chat_llm(temperature=0.1)
    
    def route(self, content: str, title: str = "") -> RoutingDecision:
        """Determine which specialist should analyze this content."""
        structured_llm = self.llm.with_structured_output(RoutingDecision)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a content router for financial analysis. Route content to specialists:

**earnings**: Quarterly/annual earnings reports, financial statements, revenue/profit data
**news**: Breaking news, events, announcements, M&A, policy changes, management changes
**market**: Market trends, sector analysis, technical analysis, broad market commentary

Choose the MOST appropriate specialist based on the primary focus of the content.
Provide reasoning for your choice."""),
            ("user", "Title: {title}\n\nContent: {content}\n\nWhich specialist should analyze this?")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({"title": title, "content": content})


class EarningsAnalyst:
    """Specialist agent for earnings analysis."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        self.llm = llm or create_chat_llm(temperature=0.2)
    
    def analyze(self, content: str) -> EarningsAnalysis:
        """Perform deep earnings analysis."""
        structured_llm = self.llm.with_structured_output(EarningsAnalysis)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert earnings analyst. Analyze financial results thoroughly.

Focus on:
- Revenue growth and trends
- Profitability (margins, EPS, operating income)
- YoY and QoQ comparisons
- Forward guidance quality
- Key performance metrics
- Investment implications

Provide actionable recommendations for investors."""),
            ("user", "Analyze this earnings content:\n\n{content}")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({"content": content})


class NewsAnalyst:
    """Specialist agent for news analysis."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        self.llm = llm or create_chat_llm(temperature=0.3)
    
    def analyze(self, content: str) -> NewsAnalysis:
        """Perform news impact analysis."""
        structured_llm = self.llm.with_structured_output(NewsAnalysis)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert news analyst specializing in financial news impact.

Analyze:
- What happened and why it matters
- Potential market impact (stocks, sectors affected)
- Stakeholder implications (companies, investors, regulators)
- Timeline of developments
- Source credibility
- Actionable insights for investors

Be objective and focus on investment implications."""),
            ("user", "Analyze this news:\n\n{content}")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({"content": content})


class MarketAnalyst:
    """Specialist agent for market analysis."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        self.llm = llm or create_chat_llm(temperature=0.3)
    
    def analyze(self, content: str) -> MarketAnalysis:
        """Perform market trend analysis."""
        structured_llm = self.llm.with_structured_output(MarketAnalysis)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert market analyst. Analyze market trends and conditions.

Evaluate:
- Overall market/sector trends
- Technical indicators and signals
- Market sentiment (bullish/bearish/neutral)
- Risk factors to watch
- Investment opportunities
- Short to medium term outlook

Provide balanced, data-driven analysis."""),
            ("user", "Analyze this market content:\n\n{content}")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({"content": content})


class RoutingWorkflow:
    """Complete routing workflow that coordinates content routing and analysis."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        """Initialize the routing workflow with all specialists."""
        self.router = ContentRouter(llm)
        self.earnings_analyst = EarningsAnalyst(llm)
        self.news_analyst = NewsAnalyst(llm)
        self.market_analyst = MarketAnalyst(llm)
    
    def process(self, content: str, title: str = "") -> Dict:
        """Route content to appropriate specialist and return analysis."""
        print("\n=== ROUTING WORKFLOW ===")
        
        # Step 1: Route the content
        print("\n[1/2] Routing content to specialist...")
        routing_decision = self.router.route(content, title)
        print(f"✓ Routed to: {routing_decision.route.upper()} analyst "
              f"(confidence: {routing_decision.confidence:.2f})")
        print(f"  Reasoning: {routing_decision.reasoning}")
        
        # Step 2: Analyze with appropriate specialist
        print(f"\n[2/2] {routing_decision.route.upper()} analyst processing...")
        
        if routing_decision.route == "earnings":
            analysis = self.earnings_analyst.analyze(content)
            analysis_type = "Earnings Analysis"
        elif routing_decision.route == "news":
            analysis = self.news_analyst.analyze(content)
            analysis_type = "News Analysis"
        else:  # market
            analysis = self.market_analyst.analyze(content)
            analysis_type = "Market Analysis"
        
        print(f"✓ {analysis_type} complete")
        print("\n=== WORKFLOW COMPLETE ===\n")
        
        return {
            "routing_decision": routing_decision,
            "analysis_type": analysis_type,
            "analysis": analysis
        }
    
    def batch_process(self, content_items: List[Dict[str, str]]) -> List[Dict]:
        """Process multiple content items, routing each appropriately."""
        results = []
        for i, item in enumerate(content_items, 1):
            print(f"\n{'='*60}")
            print(f"Processing item {i}/{len(content_items)}")
            print(f"{'='*60}")
            
            result = self.process(
                content=item.get("content", ""),
                title=item.get("title", "")
            )
            results.append(result)
        
        return results


print("✓ Routing workflow classes defined")

### Demo: Routing Workflow

In [ ]:
routing_workflow = RoutingWorkflow()

# Sample content of different types
test_content = [
    {
        "title": "Apple Reports Q1 Earnings Beat",
        "content": """
        Apple Inc. reported fiscal Q1 results with revenue of $119.6 billion, up 2% YoY,
        beating estimates of $118.3 billion. iPhone revenue reached $69.7 billion, up 6%.
        Services revenue hit a record $23.1 billion. EPS came in at $2.18 vs $2.10 expected.
        Gross margin improved to 45.9% from 43.0% last year. CEO Tim Cook noted strong
        demand in emerging markets. The company announced a 4% dividend increase.
        """
    },
    {
        "title": "Tech Sector Shows Bullish Momentum",
        "content": """
        Technology stocks continued their uptrend with the Nasdaq composite gaining 15%
        YTD. Strong momentum in AI-related names is driving the rally. The tech sector
        RSI stands at 68, approaching overbought territory. Volume has been above average,
        confirming the trend. Support levels have held at the 50-day moving average.
        Sector rotation shows investors favoring growth over value.
        """
    }
]

results = routing_workflow.batch_process(test_content)

# Display summary
print("\n" + "="*60)
print("ROUTING SUMMARY")
print("="*60)
for i, result in enumerate(results, 1):
    print(f"\nContent {i}: {test_content[i-1]['title']}")
    print(f"  → Routed to: {result['routing_decision'].route}")
    print(f"  → Analysis type: {result['analysis_type']}")

## 7. Workflow Pattern 3: Evaluator-Optimizer

Generate → Evaluate → Refine → Re-evaluate (Iterative Self-Improvement)

In [ ]:
class InvestmentAnalyzer:
    """Generates initial investment analysis (the generator)."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        self.llm = llm or create_chat_llm(temperature=0.4)
    
    def generate(self, ticker: str, company_data: str) -> InvestmentAnalysis:
        """Generate initial investment analysis."""
        structured_llm = self.llm.with_structured_output(InvestmentAnalysis)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an investment analyst. Generate a comprehensive stock analysis.

Include:
- Clear Buy/Hold/Sell recommendation
- Price target with justification
- Investment thesis (why this stock?)
- Key catalysts (what will drive the stock?)
- Risks (what could go wrong?)
- Financial highlights (metrics, trends)
- Conclusion

Base your analysis on the provided data. Be specific and actionable."""),
            ("user", "Ticker: {ticker}\n\nCompany Data:\n{company_data}\n\nProvide your analysis:")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({"ticker": ticker, "company_data": company_data})


class AnalysisEvaluator:
    """Evaluates quality of investment analysis (the evaluator)."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        self.llm = llm or create_chat_llm(temperature=0.2)
    
    def evaluate(self, analysis: InvestmentAnalysis, original_data: str) -> QualityEvaluation:
        """Evaluate the quality of an investment analysis."""
        structured_llm = self.llm.with_structured_output(QualityEvaluation)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a senior investment analyst reviewing a junior analyst's work.
Evaluate the analysis critically but constructively.

Score each dimension 0-100:
- Completeness: Are all key aspects covered?
- Accuracy: Is the logic sound? Are claims supported?
- Actionability: Can an investor act on this?

Identify:
- Strengths (what's done well)
- Weaknesses (what's lacking)
- Missing elements (what's not covered)
- Specific improvements needed

An analysis is acceptable (is_acceptable=true) only if overall_score >= 75.
Be thorough and specific in your feedback."""),
            ("user", """Original Data:
{original_data}

Analysis to Evaluate:
Ticker: {ticker}
Recommendation: {recommendation}
Target Price: ${target_price}

Investment Thesis:
{investment_thesis}

Key Catalysts:
{catalysts}

Risks:
{risks}

Financial Highlights:
{financials}

Conclusion:
{conclusion}

Provide your evaluation:""")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({
            "original_data": original_data,
            "ticker": analysis.ticker,
            "recommendation": analysis.recommendation,
            "target_price": analysis.target_price,
            "investment_thesis": analysis.investment_thesis,
            "catalysts": "\n".join(f"- {c}" for c in analysis.key_catalysts),
            "risks": "\n".join(f"- {r}" for r in analysis.risks),
            "financials": analysis.financial_highlights,
            "conclusion": analysis.conclusion
        })


class AnalysisOptimizer:
    """Refines analysis based on evaluation feedback (the optimizer)."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None):
        self.llm = llm or create_chat_llm(temperature=0.3)
    
    def refine(
        self,
        original_analysis: InvestmentAnalysis,
        evaluation: QualityEvaluation,
        company_data: str
    ) -> InvestmentAnalysis:
        """Refine analysis based on evaluation feedback."""
        structured_llm = self.llm.with_structured_output(InvestmentAnalysis)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are refining an investment analysis based on expert feedback.

Your task:
1. Address all weaknesses identified
2. Add missing elements
3. Implement improvement suggestions
4. Expand areas that need more detail
5. Clarify ambiguous points

Keep the strengths, fix the weaknesses. Make the analysis significantly better."""),
            ("user", """Original Analysis:
Ticker: {ticker}
Recommendation: {recommendation}
Target Price: ${target_price}
Investment Thesis: {investment_thesis}

Evaluation Feedback:
Overall Score: {overall_score}/100
Strengths: {strengths}
Weaknesses: {weaknesses}
Missing Elements: {missing_elements}

Specific Feedback:
{specific_feedback}

Improvement Suggestions:
{improvements}

Company Data (for reference):
{company_data}

Provide a refined, improved analysis:""")
        ])
        
        chain = prompt | structured_llm
        return chain.invoke({
            "ticker": original_analysis.ticker,
            "recommendation": original_analysis.recommendation,
            "target_price": original_analysis.target_price,
            "investment_thesis": original_analysis.investment_thesis,
            "overall_score": evaluation.overall_score,
            "strengths": ", ".join(evaluation.strengths),
            "weaknesses": ", ".join(evaluation.weaknesses),
            "missing_elements": ", ".join(evaluation.missing_elements),
            "specific_feedback": evaluation.specific_feedback,
            "improvements": "\n".join(f"- {s}" for s in evaluation.improvement_suggestions),
            "company_data": company_data
        })


class EvaluatorOptimizerWorkflow:
    """Complete evaluator-optimizer workflow with iterative refinement."""
    
    def __init__(self, llm: Optional[ChatOpenAI] = None, max_iterations: int = 3):
        self.analyzer = InvestmentAnalyzer(llm)
        self.evaluator = AnalysisEvaluator(llm)
        self.optimizer = AnalysisOptimizer(llm)
        self.max_iterations = max_iterations
        self.quality_threshold = 75.0
    
    def run(self, ticker: str, company_data: str, verbose: bool = True) -> Dict:
        """Execute the complete evaluator-optimizer workflow."""
        if verbose:
            print("\n=== EVALUATOR-OPTIMIZER WORKFLOW ===")
        
        iteration_history = []
        current_analysis = None
        current_evaluation = None
        
        for iteration in range(1, self.max_iterations + 1):
            if verbose:
                print(f"\n--- Iteration {iteration}/{self.max_iterations} ---")
            
            # Generate or refine analysis
            if iteration == 1:
                if verbose:
                    print("\n[1] Generating initial analysis...")
                current_analysis = self.analyzer.generate(ticker, company_data)
                if verbose:
                    print(f"✓ Initial analysis complete: {current_analysis.recommendation} "
                          f"recommendation, ${current_analysis.target_price} target")
            else:
                if verbose:
                    print(f"\n[1] Refining analysis based on feedback...")
                current_analysis = self.optimizer.refine(
                    current_analysis,
                    current_evaluation,
                    company_data
                )
                if verbose:
                    print(f"✓ Refined analysis: {current_analysis.recommendation} "
                          f"recommendation, ${current_analysis.target_price} target")
            
            # Evaluate
            if verbose:
                print(f"\n[2] Evaluating analysis quality...")
            current_evaluation = self.evaluator.evaluate(current_analysis, company_data)
            
            if verbose:
                print(f"✓ Quality Score: {current_evaluation.overall_score:.1f}/100")
                print(f"  - Completeness: {current_evaluation.completeness_score:.1f}")
                print(f"  - Accuracy: {current_evaluation.accuracy_score:.1f}")
                print(f"  - Actionability: {current_evaluation.actionability_score:.1f}")
                print(f"  - Acceptable: {current_evaluation.is_acceptable}")
            
            # Store iteration
            iteration_history.append({
                "iteration": iteration,
                "analysis": current_analysis,
                "evaluation": current_evaluation
            })
            
            # Check if we've met quality threshold
            if current_evaluation.overall_score >= self.quality_threshold:
                if verbose:
                    print(f"\n✓ Quality threshold met ({self.quality_threshold})! "
                          f"Analysis accepted.")
                break
            
            if iteration < self.max_iterations:
                if verbose:
                    print(f"\n  Quality below threshold. Refining...")
            else:
                if verbose:
                    print(f"\n⚠ Max iterations reached. Using best available analysis.")
        
        if verbose:
            print("\n=== WORKFLOW COMPLETE ===\n")
        
        return {
            "final_analysis": current_analysis,
            "final_evaluation": current_evaluation,
            "iteration_history": iteration_history,
            "iterations_performed": len(iteration_history),
            "quality_threshold_met": current_evaluation.overall_score >= self.quality_threshold
        }


print("✓ Evaluator-Optimizer workflow classes defined")

### Demo: Evaluator-Optimizer

In [ ]:
sample_data = """
Tesla Inc. (TSLA)

Recent Financial Performance:
- Q3 2024 Revenue: $25.2B (+8% YoY)
- Automotive Revenue: $20.0B
- Energy & Services: $5.2B (+20% YoY)
- Net Income: $1.85B
- EPS: $0.53 (beat estimates of $0.46)
- Operating Margin: 7.6% (down from 11.0% last year)
- Free Cash Flow: $1.3B

Recent Developments:
- Price cuts across major models to stimulate demand
- Cybertruck production ramping up, now at 1,000 units/week
- Energy storage deployments up 73% YoY
- FSD (Full Self-Driving) subscription revenue growing
- Opening Gigafactory in Mexico delayed to 2025

Market Metrics:
- Current Price: $242
- 52-week range: $152 - $299
- P/E Ratio: 75
- Market Cap: $765B

Industry Context:
- EV market growth slowing (was 40% YoY, now 15%)
- Increased competition from legacy automakers
- Federal EV tax credits under review
"""

eo_workflow = EvaluatorOptimizerWorkflow(max_iterations=3)
result = eo_workflow.run("TSLA", sample_data, verbose=True)

# Display final results
print("\n" + "="*60)
print("FINAL ANALYSIS RESULTS")
print("="*60)

final = result["final_analysis"]
eval_final = result["final_evaluation"]

print(f"\nTicker: {final.ticker}")
print(f"Recommendation: {final.recommendation}")
print(f"Target Price: ${final.target_price}")
print(f"\nInvestment Thesis:\n{final.investment_thesis}")
print(f"\nKey Catalysts:")
for catalyst in final.key_catalysts:
    print(f"  • {catalyst}")
print(f"\nKey Risks:")
for risk in final.risks:
    print(f"  • {risk}")

print(f"\n{'-'*60}")
print("QUALITY METRICS")
print(f"{'-'*60}")
print(f"Final Score: {eval_final.overall_score:.1f}/100")
print(f"Iterations: {result['iterations_performed']}")
print(f"Threshold Met: {result['quality_threshold_met']}")

print(f"\nScore Progression:")
for item in result["iteration_history"]:
    score = item["evaluation"].overall_score
    print(f"  Iteration {item['iteration']}: {score:.1f}/100")

## 8. Autonomous Research Agent

Integrates all three workflow patterns into a complete autonomous agent that:
1. Plans research steps
2. Dynamically uses tools
3. Self-reflects on quality
4. Learns across runs

In [ ]:
class AutonomousResearchAgent:
    """Autonomous agent that researches stocks with planning, tool use, and self-reflection."""
    
    def __init__(
        self,
        llm: Optional[ChatOpenAI] = None,
        memory_file: str = MEMORY_FILE
    ):
        self.llm = llm or create_chat_llm(temperature=0.3)
        self.memory_file = memory_file
        
        # Initialize components
        self.kg_builder = KnowledgeGraphBuilder()
        self.fetcher = FinancialDataFetcher()
        self.chaining_workflow = PromptChainWorkflow(llm)
        self.routing_workflow = RoutingWorkflow(llm)
        self.eo_workflow = EvaluatorOptimizerWorkflow(llm, max_iterations=2)
    
    def step1_plan_research(self, ticker: str, user_context: str = "") -> ResearchPlan:
        """AGENT FUNCTION 1: Plans research steps autonomously."""
        print("\n=== STEP 1: PLANNING RESEARCH ===")
        
        # Load past learnings
        past_learnings = self._load_memory()
        
        structured_llm = self.llm.with_structured_output(ResearchPlan)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an autonomous investment research agent. Plan comprehensive research.

Your research plan should:
1. Break down the research into clear, ordered steps
2. Identify required data sources
3. List key questions that must be answered
4. Estimate research complexity

Consider past learnings to improve your approach:
{past_learnings}

Be thorough, systematic, and strategic."""),
            ("user", "Ticker: {ticker}\n\nUser Context: {user_context}\n\nCreate a research plan:")
        ])
        
        chain = prompt | structured_llm
        plan = chain.invoke({
            "ticker": ticker,
            "user_context": user_context or "General investment analysis",
            "past_learnings": past_learnings
        })
        
        print(f"✓ Research plan created with {len(plan.research_steps)} steps")
        print(f"  Complexity: {plan.estimated_complexity}")
        print(f"  Data sources: {', '.join(plan.data_sources_needed)}")
        
        return plan
    
    def step2_execute_research(self, plan: ResearchPlan) -> Dict:
        """AGENT FUNCTION 2: Uses tools dynamically based on plan."""
        print("\n=== STEP 2: EXECUTING RESEARCH ===")
        results = {}
        
        # Tool 1: Build knowledge graph
        if "knowledge graph" in " ".join(plan.data_sources_needed).lower() or \
           "entity extraction" in " ".join(plan.data_sources_needed).lower():
            
            print("\n[Tool: Knowledge Graph] Building entity graph...")
            graph = self.kg_builder.expand_from_seed(
                plan.ticker,
                seed_label="STOCK_SYMBOL",
                depth=2
            )
            results["knowledge_graph"] = self.kg_builder.get_graph_summary()
            print(f"✓ Knowledge graph: {graph.number_of_nodes()} nodes")
        
        # Tool 2: Fetch and analyze news
        if "news" in " ".join(plan.data_sources_needed).lower():
            print("\n[Tool: News Analysis] Fetching recent news...")
            news_data = self.fetcher.fetch_yahoo_news(plan.ticker)
            
            if news_data and len(news_data) > 100:
                print("[Workflow: Prompt Chaining] Processing news...")
                news_summary = self.chaining_workflow.run(news_data, source="Yahoo Finance")
                results["news_analysis"] = {
                    "summary": news_summary.executive_summary,
                    "key_points": news_summary.key_points,
                    "category": news_summary.classification.category,
                    "sentiment": news_summary.classification.sentiment
                }
                print(f"✓ News analyzed: {news_summary.classification.category} / {news_summary.classification.sentiment}")
        
        # Tool 3: Route to specialized analyzer
        if "earnings" in " ".join(plan.data_sources_needed).lower() or \
           "market analysis" in " ".join(plan.data_sources_needed).lower():
            
            print("\n[Workflow: Routing] Routing to specialist...")
            content = self.fetcher.fetch_alpha_vantage_quote(plan.ticker)
            
            if content and len(content) > 50:
                routing_result = self.routing_workflow.process(
                    content,
                    title=f"{plan.ticker} Financial Data"
                )
                results["specialized_analysis"] = {
                    "route": routing_result["routing_decision"].route,
                    "analysis_type": routing_result["analysis_type"]
                }
                print(f"✓ Routed to {routing_result['routing_decision'].route} analyst")
        
        # Tool 4: Generate comprehensive investment analysis
        print("\n[Workflow: Evaluator-Optimizer] Generating investment analysis...")
        
        compiled_data = self._compile_data_for_analysis(plan.ticker, results)
        
        eo_result = self.eo_workflow.run(
            plan.ticker,
            compiled_data,
            verbose=False
        )
        
        results["investment_analysis"] = {
            "recommendation": eo_result["final_analysis"].recommendation,
            "target_price": eo_result["final_analysis"].target_price,
            "thesis": eo_result["final_analysis"].investment_thesis,
            "quality_score": eo_result["final_evaluation"].overall_score,
            "iterations": eo_result["iterations_performed"]
        }
        
        print(f"✓ Investment analysis: {eo_result['final_analysis'].recommendation} "
              f"(quality: {eo_result['final_evaluation'].overall_score:.1f}/100)")
        
        return results
    
    def step3_reflect_on_quality(self, plan: ResearchPlan, results: Dict) -> ResearchReflection:
        """AGENT FUNCTION 3: Self-reflects on research quality."""
        print("\n=== STEP 3: SELF-REFLECTION ===")
        
        structured_llm = self.llm.with_structured_output(ResearchReflection)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are reflecting on the quality of your own research. Be critical and honest.

Evaluate:
- Completeness: Did we answer all key questions?
- Confidence: How confident are we in the findings?
- Data Quality: Were the data sources reliable and sufficient?

Identify:
- Strengths: What did we do well?
- Gaps: What's missing or incomplete?
- Reliability concerns: Any data quality issues?

Provide honest self-assessment and concrete improvement recommendations."""),
            ("user", """Research Plan:
Ticker: {ticker}
Steps: {steps}
Key Questions: {questions}

Research Results:
{results}

Reflect on the quality of this research:""")
        ])
        
        chain = prompt | structured_llm
        reflection = chain.invoke({
            "ticker": plan.ticker,
            "steps": "\n".join(f"- {step}" for step in plan.research_steps),
            "questions": "\n".join(f"- {q}" for q in plan.key_questions),
            "results": str(results)[:3000]  # Limit length
        })
        
        print(f"✓ Self-reflection complete")
        print(f"  Completeness: {reflection.completeness_score:.1f}/100")
        print(f"  Confidence: {reflection.confidence_score:.1f}/100")
        print(f"  Data Quality: {reflection.data_quality_score:.1f}/100")
        print(f"  Gaps identified: {len(reflection.gaps)}")
        
        return reflection
    
    def step4_learn_from_run(self, reflection: ResearchReflection):
        """AGENT FUNCTION 4: Learns across runs."""
        print("\n=== STEP 4: LEARNING ===")
        
        learning_entry = f"""
Run: {reflection.overall_assessment}
Improvements to apply:
{chr(10).join(f"- {rec}" for rec in reflection.improvement_recommendations)}
Gaps to avoid:
{chr(10).join(f"- {gap}" for gap in reflection.gaps)}
---
"""
        
        self._save_to_memory(learning_entry)
        print(f"✓ Learnings saved to {self.memory_file}")
        print(f"  {len(reflection.improvement_recommendations)} improvement recommendations stored")
    
    def research_stock(self, ticker: str, user_context: str = "") -> ResearchResult:
        """Execute complete autonomous research workflow."""
        import time
        start_time = time.time()
        
        print("\n" + "="*80)
        print(f" AUTONOMOUS RESEARCH AGENT: {ticker}")
        print("="*80)
        
        # Step 1: Plan
        plan = self.step1_plan_research(ticker, user_context)
        
        # Step 2: Execute with dynamic tool selection
        results = self.step2_execute_research(plan)
        
        # Step 3: Self-reflect
        reflection = self.step3_reflect_on_quality(plan, results)
        
        # Step 4: Learn
        self.step4_learn_from_run(reflection)
        
        execution_time = time.time() - start_time
        
        print("\n" + "="*80)
        print(f" RESEARCH COMPLETE ({execution_time:.1f}s)")
        print("="*80 + "\n")
        
        return ResearchResult(
            ticker=ticker,
            plan=plan,
            knowledge_graph_summary=results.get("knowledge_graph", "Not generated"),
            news_analysis=results.get("news_analysis"),
            specialized_analysis=results.get("specialized_analysis"),
            investment_analysis=results.get("investment_analysis"),
            reflection=reflection,
            execution_time_seconds=execution_time
        )
    
    def _compile_data_for_analysis(self, ticker: str, results: Dict) -> str:
        """Compile research results into formatted text for analysis."""
        sections = [f"{ticker} Research Data\n"]
        
        if "knowledge_graph" in results:
            sections.append(f"Knowledge Graph:\n{results['knowledge_graph']}\n")
        
        if "news_analysis" in results:
            news = results["news_analysis"]
            sections.append(f"News Analysis:\n")
            sections.append(f"Summary: {news.get('summary', 'N/A')}\n")
            sections.append(f"Sentiment: {news.get('sentiment', 'N/A')}\n")
        
        # Add fetched financial data
        quote_data = self.fetcher.fetch_alpha_vantage_quote(ticker)
        if quote_data:
            sections.append(f"\n{quote_data}\n")
        
        return "\n".join(sections)
    
    def _load_memory(self) -> str:
        """Load past learnings from memory file."""
        try:
            with open(self.memory_file, 'r') as f:
                content = f.read()
                return content[-2000:] if content else "No past learnings yet."
        except FileNotFoundError:
            return "No past learnings yet."
    
    def _save_to_memory(self, entry: str):
        """Append learning entry to memory file."""
        with open(self.memory_file, 'a') as f:
            f.write(entry)


print("✓ AutonomousResearchAgent class defined")

## 9. Interactive Demonstration

Run the autonomous research agent on a stock ticker

In [ ]:
# Initialize the autonomous research agent
agent = AutonomousResearchAgent()

# Research a stock (you can change the ticker and context)
ticker = "AAPL"  # Change this to any stock ticker
context = "Looking for long-term growth potential in tech sector"

result = agent.research_stock(ticker=ticker, user_context=context)

### Display Research Results

In [ ]:
print("\n" + "="*80)
print(" RESEARCH SUMMARY")
print("="*80)

print(f"\nTicker: {result.ticker}")
print(f"Execution Time: {result.execution_time_seconds:.1f} seconds")

print(f"\n--- Research Plan ---")
for i, step in enumerate(result.plan.research_steps, 1):
    print(f"  {i}. {step}")

print(f"\nEstimated Complexity: {result.plan.estimated_complexity}")
print(f"Data Sources Used: {', '.join(result.plan.data_sources_needed)}")

if result.investment_analysis:
    print(f"\n--- Investment Analysis ---")
    print(f"Recommendation: {result.investment_analysis['recommendation']}")
    print(f"Target Price: ${result.investment_analysis['target_price']}")
    print(f"Quality Score: {result.investment_analysis['quality_score']:.1f}/100")
    print(f"Refinement Iterations: {result.investment_analysis['iterations']}")

if result.news_analysis:
    print(f"\n--- News Analysis ---")
    print(f"Summary: {result.news_analysis['summary']}")
    print(f"Sentiment: {result.news_analysis['sentiment']}")
    print(f"Category: {result.news_analysis['category']}")

print(f"\n--- Self-Reflection ---")
print(f"Completeness: {result.reflection.completeness_score:.1f}/100")
print(f"Confidence: {result.reflection.confidence_score:.1f}/100")
print(f"Data Quality: {result.reflection.data_quality_score:.1f}/100")

if result.reflection.gaps:
    print(f"\nIdentified Gaps:")
    for gap in result.reflection.gaps[:3]:
        print(f"  • {gap}")

if result.reflection.improvement_recommendations:
    print(f"\nImprovement Recommendations:")
    for rec in result.reflection.improvement_recommendations[:3]:
        print(f"  • {rec}")

print(f"\nOverall Assessment: {result.reflection.overall_assessment}")
print("\n" + "="*80 + "\n")

## Conclusion

This notebook demonstrates:

1. **Three Agentic Workflow Patterns**:
   - Prompt Chaining for sequential processing
   - Routing for intelligent task distribution
   - Evaluator-Optimizer for iterative refinement

2. **Autonomous Agent Capabilities**:
   - Planning research steps
   - Dynamic tool selection and execution
   - Self-reflection on output quality
   - Learning across runs

3. **Integration of Multiple Components**:
   - Knowledge graph construction
   - Financial data fetching from multiple sources
   - LLM-powered analysis and reasoning
   - Structured output with Pydantic models

You can modify the code cells above to:
- Research different stock tickers
- Adjust workflow parameters
- Experiment with different prompts
- Add new data sources or analysis methods